In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
import re
from scipy.signal import butter, filtfilt

##### Gathering data

In [ ]:
mmmp_dir = 'directory_PATH'
fma_pick_key_control = glob(os.path.join(mmmp_dir, '**/Picking_Key*_C[0-9]*.tsv'), recursive=True)

mmmp_dir = 'directory_PATH'
fma_pick_key_pwp = glob(os.path.join(mmmp_dir, '**/Picking_Key*_P[0-9]*.tsv'), recursive=True)

In [ ]:
# Data passes thorugh low-pass butterworth filter of 4th order
def butter_lowpass_filter(data, cutoff_frequency=10, sampling_rate=100, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data, axis=0, padlen=0)
    return filtered_data

##### Reading file function

In [ ]:
NAN_special = -999999

def read_file(file):
    # Load the motion capture data into a Pandas DataFrame
    df = pd.read_csv(file,sep = '\t',skiprows=11)
    df = df.fillna(NAN_special)
    df.columns = df.columns.str.strip() #eliminating unecesary spaces in column titles
    time =  df.loc[:, "Time"].to_numpy()
    
    marker_list = ['RSHO X', 'RSHO Y', 'RSHO Z', 'LSHO X', 'LSHO Y','LSHO Z', 
                   'LWRA X', 'LWRA Y', 'LWRA Z', 'RWRA X', 'RWRA Y', 'RWRA Z',
                   'STRN X', 'STRN Y', 'STRN Z',
                   'LASI X', 'LASI Y', 'LASI Z', 'RASI X', 'RASI Y', 'RASI Z',
                   'LTOE X', 'LTOE Y', 'LTOE Z','RTOE X', 'RTOE Y', 'RTOE Z',
                  'LANK X', 'LANK Y', 'LANK Z','RANK X', 'RANK Y', 'RANK Z',
                  'LHEE X', 'LHEE Y', 'LHEE Z','RHEE X', 'RHEE Y', 'RHEE Z',
                  'LKNE X', 'LKNE Y', 'LKNE Z','RKNE X', 'RKNE Y', 'RKNE Z',
                  'Key X', 'Key Y', 'Key Z']

    # create empty arrays for each marker
    RSHO = np.zeros((len(time), 3))
    LSHO = np.zeros((len(time), 3))
    LSTC = np.zeros((len(time), 3))
    RBAK = np.zeros((len(time), 3))
    LWRA = np.zeros((len(time), 3))
    RWRA = np.zeros((len(time), 3))
    LBHD = np.zeros((len(time), 3))
    RBHD = np.zeros((len(time), 3))
    STRN = np.zeros((len(time), 3))
    LASI = np.zeros((len(time), 3))
    RASI = np.zeros((len(time), 3))
    LPSI = np.zeros((len(time), 3))
    RPSI = np.zeros((len(time), 3))
    RTOE = np.zeros((len(time), 3))
    LTOE = np.zeros((len(time), 3))
    LANK = np.zeros((len(time), 3))
    RANK = np.zeros((len(time), 3))
    LHEE = np.zeros((len(time), 3))
    RHEE = np.zeros((len(time), 3))
    LKNE = np.zeros((len(time), 3))
    RKNE = np.zeros((len(time), 3))
    Key = np.zeros((len(time), 3))
    
    flag = False # this flAg will help us identify the files that are incomplete later 
    missing_markers = [] # this is a list to collect the missing markers in the file
    
    for col_name in marker_list:
        if col_name in df.columns:
            
            if col_name.startswith('RSHO'):
                RSHO = df.loc[:, ['RSHO X', 'RSHO Y', 'RSHO Z']].to_numpy()
                RSHO = butter_lowpass_filter(RSHO)
            elif col_name.startswith('LSHO'):
                LSHO = df.loc[:, ['LSHO X', 'LSHO Y', 'LSHO Z']].to_numpy()
                LSHO = butter_lowpass_filter(LSHO)

            elif col_name.startswith('LWRA'):
                LWRA = df.loc[:, ['LWRA X', 'LWRA Y', 'LWRA Z']].to_numpy()
                LWRA = butter_lowpass_filter(LWRA)
            elif col_name.startswith('RWRA'):
                RWRA = df.loc[:, ['RWRA X', 'RWRA Y', 'RWRA Z']].to_numpy()
                RWRA = butter_lowpass_filter(RWRA)

            elif col_name.startswith('STRN'):
                STRN = df.loc[:, ['STRN X', 'STRN Y', 'STRN Z']].to_numpy()
                STRN = butter_lowpass_filter(STRN)
            elif col_name.startswith('LASI'):
                LASI = df.loc[:, ['LASI X', 'LASI Y', 'LASI Z']].to_numpy()
                LASI = butter_lowpass_filter(LASI)
            elif col_name.startswith('RASI'):
                RASI = df.loc[:, ['RASI X', 'RASI Y', 'RASI Z']].to_numpy()
                RASI = butter_lowpass_filter(RASI)
            elif col_name.startswith('RTOE'):
                RTOE = df.loc[:, ['RTOE X', 'RTOE Y', 'RTOE Z']].to_numpy()
                RTOE = butter_lowpass_filter(RTOE)
            elif col_name.startswith('LTOE'):
                LTOE = df.loc[:, ['LTOE X', 'LTOE Y', 'LTOE Z']].to_numpy()
                LTOE = butter_lowpass_filter(LTOE)
            elif col_name.startswith('LANK'):
                LANK = df.loc[:, ['LANK X', 'LANK Y', 'LANK Z']].to_numpy()
                LANK = butter_lowpass_filter(LANK)
            elif col_name.startswith('RANK'):
                RANK = df.loc[:, ['RANK X', 'RANK Y', 'RANK Z']].to_numpy()
                RANK = butter_lowpass_filter(RANK)
            elif col_name.startswith('LHEE'):
                LHEE = df.loc[:, ['LHEE X', 'LHEE Y', 'LHEE Z']].to_numpy()
                LHEE = butter_lowpass_filter(LHEE)
            elif col_name.startswith('RHEE'):
                RHEE = df.loc[:, ['RHEE X', 'RHEE Y', 'RHEE Z']].to_numpy()
                RHEE = butter_lowpass_filter(RHEE)
            elif col_name.startswith('LKNE'):
                LKNE = df.loc[:, ['LKNE X', 'LKNE Y', 'LKNE Z']].to_numpy()
                LKNE = butter_lowpass_filter(LKNE)
            elif col_name.startswith('RKNE'):
                RKNE = df.loc[:, ['RKNE X', 'RKNE Y', 'RKNE Z']].to_numpy()
                RKNE = butter_lowpass_filter(RKNE)
            elif col_name.startswith('Key'):
                Key = df.loc[:, ['Key X', 'Key Y', 'Key Z']].to_numpy()
                Key = butter_lowpass_filter(Key)
        else:
            flag = True
            missing_markers.append(col_name)
#             print('File: {} , does not have marker {}'.format(file, col_name))
    print(file,missing_markers)
    
    
    
    return flag,missing_markers,time,RSHO,LSHO,STRN,LASI,RASI,RTOE,LTOE,LHEE,RHEE,LKNE,RKNE,RWRA,LWRA,Key

In [ ]:
flag,missing_markers,time,RSHO,LSHO,STRN,LASI,RASI,RTOE,LTOE,LHEE,RHEE,LKNE,RKNE,RWRA,LWRA,Key = read_file(fma_pick_key_pwp[7])

# Peak flexion angle of pelvis

Peak flexion angle of the pelvis, will be the min angle formed between the trunk-pelvis and trunk-knees vector. 
- If Angle -> 0 = straight body
- If angle -> 180 = bent body

In [ ]:
def calculate_pelvis_flexion_angle(left_pelvis, right_pelvis, sternum, left_knee, right_knee):
    """
    Calculate the average angle of pelvis flexion based on the position of pelvic markers, sternum marker, and knee markers.

    Parameters:
    - left_pelvis (numpy.ndarray): Position vector representing the left pelvis marker.
    - right_pelvis (numpy.ndarray): Position vector representing the right pelvis marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.
    - left_knee (numpy.ndarray): Position vector representing the left knee marker.
    - right_knee (numpy.ndarray): Position vector representing the right knee marker.

    Returns:
    - float: The average angle of pelvis flexion, in degrees.
    """
    
    hip_sternum_vec1 = sternum - left_pelvis
    hip_sternum_vec2 = sternum - right_pelvis

    
    hip_knee_vec1 = left_knee - left_pelvis
    hip_knee_vec2 = right_knee - right_pelvis

    
    dot_product1 = np.sum(hip_sternum_vec1 * hip_knee_vec1, axis=1)
    dot_product2 = np.sum(hip_sternum_vec2 * hip_knee_vec2, axis=1)

    norm1 = np.linalg.norm(hip_sternum_vec1, axis=1) * np.linalg.norm(hip_knee_vec1, axis=1)
    norm2 = np.linalg.norm(hip_sternum_vec2, axis=1) * np.linalg.norm(hip_knee_vec2, axis=1)

  
    cos_angle1 = np.clip(dot_product1 / norm1, -1.0, 1.0)
    cos_angle2 = np.clip(dot_product2 / norm2, -1.0, 1.0)

    angle1 = np.arccos(cos_angle1)
    angle2 = np.arccos(cos_angle2)

    angle_degrees1 = 180-np.degrees(angle1)
    angle_degrees2 = 180-np.degrees(angle2)
    
    avg_angle = np.mean([angle_degrees1, angle_degrees2],axis=0)
    return np.max(avg_angle) #,avg_angle

# Example usage:
peak_flexion_angle = calculate_pelvis_flexion_angle(LASI, RASI, STRN, LKNE, RKNE)
print(peak_flexion_angle)

#  Peak flexion angle of knees

Peak flexion angle of the knees, will be the min angle formed between the knee-pelvis and knee-heel vector. 
- If Angle -> 180 = straight leg
- If angle -> 0 bent knee

In [ ]:
def calculate_knee_flexion_angle(left_pelvis, right_pelvis, left_knee, right_knee,left_heel,right_heel):
    """
    Calculate the average angle of knee flexion based on the position of pelvic markers, knee markers, and heel markers.

    Parameters:
    - left_pelvis (numpy.ndarray): Position vector representing the left pelvis marker.
    - right_pelvis (numpy.ndarray): Position vector representing the right pelvis marker.
    - left_knee (numpy.ndarray): Position vector representing the left knee marker.
    - right_knee (numpy.ndarray): Position vector representing the right knee marker.
    - left_heel (numpy.ndarray): Position vector representing the left heel marker.
    - right_heel (numpy.ndarray): Position vector representing the right heel marker.

    Returns:
    - float: The average angle of knee flexion, in degrees.
    """
    
    hip_knee_vec1 = left_pelvis - left_knee
    hip_knee_vec2 = right_pelvis - right_knee
    
    knee_heel_vec1 = left_heel - left_knee
    knee_heel_vec2 = right_heel - right_knee

    
    dot_product1 = np.sum(hip_knee_vec1 * knee_heel_vec1, axis=1)
    dot_product2 = np.sum(hip_knee_vec2 * knee_heel_vec2, axis=1)

  
    norm1 = np.linalg.norm(hip_knee_vec1, axis=1) * np.linalg.norm(knee_heel_vec1, axis=1)
    norm2 = np.linalg.norm(hip_knee_vec2, axis=1) * np.linalg.norm(knee_heel_vec2, axis=1)


    cos_angle1 = np.clip(dot_product1 / norm1, -1.0, 1.0)
    cos_angle2 = np.clip(dot_product2 / norm2, -1.0, 1.0)

    angle1 = np.arccos(cos_angle1)
    angle2 = np.arccos(cos_angle2)

  
    angle_degrees1 = np.degrees(angle1)
    angle_degrees2 = np.degrees(angle2)
    
#     plt.plot(angle_degrees1,label='left knee angle')
#     plt.plot(angle_degrees2,label='right knee angle')
#     plt.legend()
  
    avg_angle = np.mean([angle_degrees1, angle_degrees2],axis=0)  # mean angle (left, right side)

    peak_flexion_angle = np.min(avg_angle)
    
    return peak_flexion_angle

# Example usage:
# peak_flexion_angle = calculate_knee_flexion_angle(LASI, RASI,LKNE, RKNE,LHEE,RHEE)
# print(peak_flexion_angle)
# # plt.plot(mean_angle)

# Foot positioning and angle between feet

Foot positioning is calculated with the middlepoints of each foot, we take the maximum magnitude of the displacement between the two.
The angle is calculated with each foot vector, considering both feet have the heel on the origin (0,0), we calculate the angle between the two vectors. 
then if:
- angle -> 0 : feet are straight, toes tend to point forward
- angle -> 180 : the toes tend to point toward the exterior 

In [ ]:
def calculate_angle_between_feet(left_toe, left_heel, right_toe, right_heel):
    """
    Calculate the angle between a pair of foot vectors.
    
    Parameters:
    left/right toe (numpy.ndarray): Toe positions (N_samples x 3).
    left/right heel  (numpy.ndarray): Heel positions (N_samples x 3).
    
    Returns:
    numpy.ndarray: max angle between right and left foot
    """
    
    # Calculate foot vectors
    left_foot_vector = left_toe - left_heel
    right_foot_vector = right_toe - right_heel
   
    left_foot_vector_norm = left_foot_vector / np.linalg.norm(left_foot_vector, axis=1)[:, np.newaxis]
    right_foot_vector_norm = right_foot_vector / np.linalg.norm(right_foot_vector, axis=1)[:, np.newaxis]

    # Calculate the angle between the vectors for each pair
    dot_products = np.einsum('ij,ij->i', left_foot_vector_norm, right_foot_vector_norm)
    angles = np.max(np.arccos(np.clip(dot_products, -1.0, 1.0)))
    angles_feet_degrees = np.degrees(angles)

    return angles_feet_degrees

def calculate_forward_displacement(left_toe, left_heel, right_toe, right_heel):
    """
    Calculate the forward displacement of one foot relative to the other.
    
    Parameters:
    left/right toe (numpy.ndarray): Toe positions (N_samples x 3).
    left/right heel  (numpy.ndarray): Heel positions (N_samples x 3).
    
    Returns:
    numpy.ndarray: max absolute forward displacement 
    """
       
    # Compute the midpoints of the feet
    midpoint_left = (left_toe + left_heel) / 2
    midpoint_right = (right_toe + right_heel) / 2
    
    displacement = midpoint_right - midpoint_left
    
    max_absolute_displacement = np.max(np.linalg.norm(displacement, axis=1)) # we calculate the magnitude since we dont care which 
                                                                # feet is in front, we just want to know how far apart they are
                                                             # from one another
    return max_absolute_displacement


angles = calculate_angle_between_feet(LTOE,LHEE,RTOE,RHEE)
displacement = calculate_forward_displacement(LTOE,LHEE,RTOE,RHEE)

print("Angles between foot vectors (in degrees):", angles)
print("Forward displacement of the right foot relative to the left foot:", displacement)


# Stance width

This code calculates the horizontal distance between corresponding markers on the left and right feet. it measures the width of the stance in the horizontal plane without considering the overall position of the feet.

In [ ]:
def calculate_stance_width(left_toe, left_heel, right_toe, right_heel):
    """
    Calculate the horizontal distance between vectors of the left and right foot.
    
     Parameters:
    left/right toe (numpy.ndarray): Toe positions (N_samples x 3).
    left/right heel  (numpy.ndarray): Heel positions (N_samples x 3).
    
    Returns:
    numpy.ndarray: max horizontal distance between corresponding markers
    """
    left_foot_vector = left_toe - left_heel
    right_foot_vector = right_toe - right_heel
    
    max_stance_width = np.max(np.abs(left_foot_vector[:, 0] - right_foot_vector[:, 0]))
    return max_stance_width

print(calculate_stance_width(LTOE,LHEE,RTOE,RHEE))

# Max Toe-off angle

Max TO is calculated for every feet, then the max between the two is taken for comparison

In [ ]:
def calculate_foot_angle(toe, heel):
    """
    Parameters:
    toe (numpy.ndarray): A 3D position vector of the toe marker (1D array with 3 elements [x, y, z]).
    heel (numpy.ndarray): A 3D position vector of the heel marker (1D array with 3 elements [x, y, z]).
    
    Returns:
    tuple:
        - theta (float): The absolute angle (in radians) between the toe and heel vectors 
          in the vertical plane, indicating the degree of foot strike or toe-off.
        - result (str): A string indicating whether the angle corresponds to a heel strike ("HS") 
          or a toe-off ("TO")
    """
    # Calculate the vector between Marker 1 and Marker 2
    vector = heel - toe

    # Calculate the hypotenuse (c)
    c = np.linalg.norm(vector)

    # Calculate z component
    z = toe[2] - heel[2]

    # Calculate the foot strike angle (θ)
    theta = np.arcsin(z / c)
    
    # Determine if it's toe-off angle (TO) or heel strike angle (HS)
    if theta > 0:
        result = "HS"  #  Heel strike angle
    else:
        result = "TO"  #  Toe-off angle
        theta = np.abs(theta)
        
    return np.degrees(theta), result

def obtain_max_TO_angle_between_feet(left_toe, left_heel, right_toe, right_heel):
    """
    Obtain the maximum toe-off (TO) angle between the left and right feet.

    This function calculates the toe-off angle for each pair of toe and heel markers on both the left and right feet.
    It then identifies the maximum toe-off angle for each foot and returns the larger of these maximum angles.
    The toe-off angle is determined by the angle between the toe and heel markers where the foot is in the toe-off position.

    Parameters:
    left/right toe (numpy.ndarray): Toe positions (N_samples x 3).
    left/right heel  (numpy.ndarray): Heel positions (N_samples x 3).

    Returns:
    float: The maximum toe-off angle between the left and right feet. This value represents the largest angle 
       where the foot is in the toe-off position among both feet.

    """
        
    left_to = pd.DataFrame(columns=['theta'])
    for marker1, marker2 in zip(left_toe, left_heel):
        theta, result = calculate_foot_angle(marker1, marker2)
        if result == 'TO':
            new_data_L_angle = {'theta': theta}
            # Add the new line to the DataFrame
            left_to.loc[len(left_to)] = new_data_L_angle


    right_to = pd.DataFrame(columns=['theta'])
    for marker1, marker2 in zip(right_toe, right_heel):
        theta, result = calculate_foot_angle(marker1, marker2)
        if result == 'TO':
            new_data_L_angle = {'theta': theta}
            # Add the new line to the DataFrame
            right_to.loc[len(right_to)] = new_data_L_angle
#     print(right_to)
    
    if len(right_to.theta.value_counts()) > 0 and len(left_to.theta.value_counts()):
        return max(right_to['theta'].max(numeric_only=True),left_to['theta'].max(numeric_only=True))
    elif len(right_to.theta.value_counts()) > 0:
        return right_to['theta'].max(numeric_only=True)
    else:
        return left_to['theta'].max(numeric_only=True)
        
#     # MAX right to
#     max_right_to = right_to['theta'].max(numeric_only=True)
#     #     print(left_to)
# #     # MAX Left to
# #     max_left_to = left_to['theta'].max(numeric_only=True)

    
#     return max(max_right_to,max_left_to)  # MAX TO between two feet
    


# plt.plot(left_to['theta'])
# plt.plot(right_to['theta'])
obtain_max_TO_angle_between_feet(LTOE,LHEE,RTOE,RHEE)

# Low reaching alignment

In [ ]:
def calculate_sagittal_plane_normal(shoulder, hip):
    """
    Calculate the normal vector to the sagittal plane defined by shoulder and hip vectors.

    Parameters:
    shoulder (numpy.ndarray): 3D positions of the shoulder markers (N_samples x 3).
    hip (numpy.ndarray): 3D positions of the hip markers (N_samples x 3).

    Returns:
    numpy.ndarray: Normal vector to the sagittal plane (N_samples x 3).
    """
    # Calculate the shoulder-hip vector
    shoulder_hip_vector = hip - shoulder

    # the y-axis is [0, 1, 0] = orthogonal to the frontal plane
    y_axis = np.array([0, 1, 0])

    # Calculate the normal vector (orthogonal) to the sagittal plane using cross product
    normal_vector = np.cross(shoulder_hip_vector, y_axis)

    # Normalize the normal vector
    normal_vector_norm = normal_vector / np.linalg.norm(normal_vector, axis=1, keepdims=True)
    
    return normal_vector_norm

def calculate_deviation_from_sagittal_plane(shoulder, wrist, sagittal_plane_normal):
    """
    Calculate how much the shoulder-wrist vector deviates from the sagittal plane.

    Parameters:
    shoulder (numpy.ndarray): 3D positions of the shoulder markers (N_samples x 3).
    wrist (numpy.ndarray): 3D positions of the wrist markers (N_samples x 3).
    sagittal_plane_normal (numpy.ndarray): Normal vector to the sagittal plane (N_samples x 3).

    Returns:
    numpy.ndarray: Angles of deviation from the sagittal plane in degrees (N_samples).
    """
    # Calculate the shoulder-wrist vector
    shoulder_wrist_vector = wrist - shoulder

    # Normalize the shoulder-wrist vector
    shoulder_wrist_norm = shoulder_wrist_vector / np.linalg.norm(shoulder_wrist_vector, axis=1, keepdims=True)

    # Calculate the dot product between the shoulder-wrist vector and the sagittal plane normal
    dot_product = np.einsum('ij,ij->i', shoulder_wrist_norm, sagittal_plane_normal) #einsum - alternative way for dot product, np.dot was not working
#     dot_product = np.dot(shoulder_wrist_norm, sagittal_plane_normal)

    # Calculate the angle in radians and convert to degrees
#     angle_radians = np.arccos(np.clip(dot_product, -1.0, 1.0))
    angle_radians = np.arccos(dot_product)
    angle_degrees = np.degrees(angle_radians)
#     plt.plot(range(len(angle_degrees)),angle_degrees)
    return angle_degrees

def return_angle_of_reaching_side(left_shoulder,left_hip,left_wrist,right_shoulder,right_hip,right_wrist,key_marker):
    # Calculate sagittal plane normals
    left_sagittal_plane_normal = calculate_sagittal_plane_normal(left_shoulder, left_hip)
    right_sagittal_plane_normal = calculate_sagittal_plane_normal(right_shoulder, right_hip)

    # Calculate deviations from sagittal plane
    left_deviation = calculate_deviation_from_sagittal_plane(left_shoulder, left_wrist, left_sagittal_plane_normal)
    right_deviation = calculate_deviation_from_sagittal_plane(right_shoulder, right_wrist, right_sagittal_plane_normal)
    
    # Key (object) position
    Key_final = key_marker[-1] #2d
    # Calculate distances from Key (object) position
    left_to_Key_distance = np.linalg.norm(left_wrist[-1][:2] - Key_final[:2])
    right_to_Key_distance = np.linalg.norm(right_wrist[-1][:2] - Key_final[:2])


    # Compare which side is reaching towards the Key (object)
    if left_to_Key_distance < right_to_Key_distance:
        reaching_side = 'Left'
        max_angle_reaching_side = 90 - np.min(left_deviation)
    elif right_to_Key_distance < left_to_Key_distance:
        reaching_side = 'Right'
        max_angle_reaching_side = 90 - np.min(right_deviation)
    else:
        reaching_side = 'Both or None'

    return reaching_side,max_angle_reaching_side
#     (LSHO, LASI)]    

reaching_side,angle_reaching_side = return_angle_of_reaching_side(LSHO,LASI,LWRA,RSHO,RASI,RWRA,Key)

# Output results
# print("Left Shoulder-Wrist Deviation from Sagittal Plane (degrees):", left_deviation)
# print("Right Shoulder-Wrist Deviation from Sagittal Plane (degrees):", right_deviation)
print(reaching_side,angle_reaching_side )

# Storing data

In [ ]:
#Create the dataframes to store the shoulder tilt and stroop angles
df_picking_key = pd.DataFrame(columns=['Subject ID','Group','TrialFMA',
                                   'Task_duration[s]','Peak_knee_flexion[deg]','Peak_pelvis_flexion[deg]',
                                      'Aperture_angle_between_feet[deg]', 'AP_distance_bewteen_feet[mm]',
                                       'Peak_Stance_width[mm]',
                                       'Max_TO_angle[deg]',
                                       'Low_reaching_arm_body_alignment[deg]'
                                      ])


In [ ]:
full_data = [fma_pick_key_control,fma_pick_key_pwp]
data_label = ['Control','PwP']

for i,group in enumerate(full_data):
    type_subject = data_label[i]
    for file in group:
                
       # Extract subject ID
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.tsv$', file).group(1)
            subject_id = 'C'+subject_id
            color = 'blue'
        else: 
            subject_id = re.search(r'_P(\d+)\.tsv$', file).group(1)
            subject_id = 'P'+subject_id


        # Extract trial ID
        trial = re.search(r'_FMA0(\d+)_', file).group(1)
        
        flag,missing_markers,time,RSHO,LSHO,STRN,LASI,RASI,RTOE,LTOE,LHEE,RHEE,LKNE,RKNE,RWRA,LWRA,Key = read_file(file)
        
        task_duration = np.max(time)
        
        if any(marker.startswith('STRN') or marker.startswith('LSHO') or marker.startswith('RSHO') for marker in missing_markers):
#             print(subject_id + "One or more required markers are missing.")
            low_reach_alignment = np.nan
        else:
            side,low_reach_alignment = return_angle_of_reaching_side(LSHO,LASI,LWRA,RSHO,RASI,RWRA,Key)
        
        if any(marker.startswith('LTOE') or marker.startswith('LHEE') or marker.startswith('RTOE') or marker.startswith('RHEE') for marker in missing_markers):
            print(subject_id + " One or more required markers are missing.")
            angle_between_feet = ap_distance_between_feet = peak_stance_width = max_TO = np.nan
 
        else:
            angle_between_feet = calculate_angle_between_feet(LTOE,LHEE,RTOE,RHEE)
            ap_distance_between_feet = calculate_forward_displacement(LTOE,LHEE,RTOE,RHEE)
            peak_stance_width = calculate_stance_width(LTOE,LHEE,RTOE,RHEE)
            max_TO = obtain_max_TO_angle_between_feet(LTOE,LHEE,RTOE,RHEE)

        if any(marker.startswith('LASI') or marker.startswith('RASI') for marker in missing_markers):
            print(subject_id + " One or more required markers are missing.")
            low_reach_alignment = peak_pelvis_flexion_angle = peak_knee_flexion_angle = np.nan
        else:
            peak_pelvis_flexion_angle = calculate_knee_flexion_angle(LASI, RASI,LKNE, RKNE,LHEE,RHEE)
            peak_knee_flexion_angle = calculate_pelvis_flexion_angle(LASI, RASI, STRN, LKNE, RKNE)
            side,low_reach_alignment = return_angle_of_reaching_side(LSHO,LASI,LWRA,RSHO,RASI,RWRA,Key)

        new_line = {'Subject ID':subject_id,'Group':type_subject,'TrialFMA':trial,
                     'Task_duration[s]':task_duration,
                    'Peak_knee_flexion[deg]':peak_knee_flexion_angle,
                    'Peak_pelvis_flexion[deg]':peak_pelvis_flexion_angle,
                      'Aperture_angle_between_feet[deg]':angle_between_feet, 
                    'AP_distance_bewteen_feet[mm]':ap_distance_between_feet,
                       'Peak_Stance_width[mm]':peak_stance_width,
                       'Max_TO_angle[deg]':max_TO,
                   'Low_reaching_arm_body_alignment[deg]':low_reach_alignment
                    }

        df_picking_key.loc[len(df_picking_key)] = new_line

In [ ]:
df_picking_key